In [3]:
# customer_model_training.py

import os
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler  # ✅ Import fixed
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

# Load and preprocess data
df = pd.read_csv("data/Passenger_Satisfaction.csv").dropna()

# Encode categorical features
le = LabelEncoder()
for col in ['Gender', 'Customer Type', 'Type of Travel', 'Class', 'satisfaction']:
    df[col] = le.fit_transform(df[col])

# Features and labels
X = df.drop(['id', 'satisfaction'], axis=1)
y = df['satisfaction']

# Feature scaling
scaler = StandardScaler()  # ✅ Fixed
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Classifiers
models = {
    "logistic_regression": LogisticRegression(max_iter=1000),
    "random_forest": RandomForestClassifier(),
    "gradient_boosting": GradientBoostingClassifier(),
    "xgboost": XGBClassifier(),
    "knn": KNeighborsClassifier()
}

# Save models
os.makedirs("customer_satisfaction_prediction/models", exist_ok=True)
joblib.dump(scaler, "customer_satisfaction_prediction/models/scaler.pkl")

for name, model in models.items():
    model.fit(X_train, y_train)
    joblib.dump(model, f"customer_satisfaction_prediction/models/{name}.pkl")

print("✅ Models and scaler saved successfully!")


✅ Models and scaler saved successfully!
